In [2]:

#Importar librerias? Pues claro que me importan!
import numpy as np
from numpy import random
from random import choices 
import os 


In [3]:

def asintomatico_o_no():
    estado = ["asintomatico", "sintomatico"]
    peso_de_decision = [0.6, 0.4]
    n = choices(estado, peso_de_decision)
    return n[0]

def probabilidad_de_muerte(edad):
    if(edad > 65):
        valoracion = [0,1]
        peso = [0.87, 0.13] #queda pendiente revisar esto 
        n = choices(valoracion, peso)
    elif(edad <= 65):
        valoracion = [0,1]
        peso = [0.99, 0.01] #probabilidad que se correspende en caso que no tenga antecentes de ningun tipo
        n = choices(valoracion, peso) #n es un vector cuyo primer elemento es la valoracion 
    return n[0]
    

In [56]:
#Objeto que simula a una persona.
class Persona:
        def __init__(self, idpersona,edad,lugaroriginal, dni):           
            self.idpersona=idpersona
            self.edad=edad
            self.estadosposibles=["sano","incubando","incubandoContagioso","sintomatico","asintomatico","muerto","inmune"]
            self.estado=self.estadosposibles[0]
            self.lugar=lugaroriginal
            self.lugaroriginal=lugaroriginal
            self.diasestado=0
            self.cambioEstado=np.inf  #Atributo que indica que dia cambiara la persona de estado.
            self.contadorInfecciones=0
            self.dni = dni
            self.edificiomuerte=-1
            
            
           
        def __str__(self):
            #Devuelve un string con la informacion util del sujeto
            return str(self.idpersona) + "," + str(self.estado) + "," + str(self.lugar) + "," +str(self.dni) + "," +str(self.edad)            

        def puede_propagar(self):
            return ((self.estado == "sintomatico") or (self.estado == "asintomatico") or (self.estado == "incubandoContagioso"))
        def susceptible(self):
            return (self.estado == "sano")
        
        def esta_sano(self):
            return (self.estado == "sano") or (self.estado == "inmune")
        
        def infectar(self,posibilidadContagio,ciudad,mediaincubacion,desvincubacion,diaactual): #media y desv covid: 7.5 y 3.5
            if self.puede_propagar():
                for i in range(len(ciudad[self.lugar])): 
                    if (ciudad[self.lugar][i].susceptible()):
                        nocontagia=random.uniform(0, 1)   #Numero al azar, si es menor que la pos. de contagio, contagiara
                        if (nocontagia <posibilidadContagio/24):
                            ciudad[self.lugar][i].estado=ciudad[self.lugar][i].estadosposibles[1]
                            ciudad[self.lugar][i].diasestado=0
                            ciudad[self.lugar][i].cambioEstado=diaactual+(random.randint(int(mediaincubacion-desvincubacion),int(mediaincubacion+desvincubacion))/2)
                           # print(ciudad[self.lugar][i].cambioEstado)
                            self.contadorInfecciones +=1
                            
        def transicionEstados(self, dia, mediaincubacion,desvincubacion,mediaduracion,desvduracion):
            #A partir de aqui se comprueba que estado posee la persona
            #Y se mira dos casos:
            #Si el dia del cambio de estado , coincide con el dia actual
            if(self.estado == "sintomatico"):
                me_voy_a_morir = probabilidad_de_muerte(self.edad)
                if(me_voy_a_morir == 1):
                    self.cambioEstado=np.inf
                    self.estado = "muerto"
            
            if(self.cambioEstado <= dia):
                if(self.estado == "incubando"):
                    self.estado = "incubandoContagioso" #se le asigna el siguiente estado  
                    self.cambioEstado = dia + random.randint(int(mediaincubacion-desvincubacion),int(mediaincubacion+desvincubacion))/2    #tiempo que pasa a inmune o muerto xD

                elif(self.estado == "incubandoContagioso"):
                    self.estado = asintomatico_o_no() #se le asigna el siguiente estado 
                    self.cambioEstado =dia+ random.randint(int(mediaduracion-desvduracion),int(mediaduracion+desvduracion)) #tiempo que pasa a inmune o muerto xD

                elif(self.estado == "sintomatico"):
                    self.cambioEstado = np.inf 
                    self.estado = "inmune"
                    '''
                      Y ahora una vez que pasa a ser asintomatico o no, se debe ver la probabilidad de que la persona se vaya 
                       a morir o no, y dependiendo de eso, se ha asignan unos dias...
                    '''

                elif(self.estado == "asintomatico"):
                    self.estado = "inmune"
                    self.cambioEstado = np.inf 
                                     
                

In [386]:

class Simulador:
        def __init__(self, numpersonas,personasedificio):           
            self.ciudad=crearciudadv0(numpersonas,personasedificio)
            self.dia=0 #se inicializa el dia con un valor de 0 
            self.hora=0 #se inicializa las horas con un valor de 0 
            self.numpersonasinicial=numpersonas 
            self.numpersonas=numpersonas
            self.numedificios=len(self.ciudad)
            self.capacidades=np.zeros(self.numedificios)
            self.CatalogoPersonas=[]
            self.cementerio=[]
            for i in range(self.numedificios-1,-1,-1):
                self.capacidades[i]=int(len(self.ciudad[i])) #establecemos las capacidades de los edificios
                for j in range (len(self.ciudad[i])):
                    self.CatalogoPersonas.append(self.ciudad[i][j])
                    
            self.capacidades=self.capacidades.astype(int)
            print(self.capacidades)
            #for i in range(len(self.CatalogoPersonas)):
            #    print(self.CatalogoPersonas[i])
            
                
           
        def __str__(self):
            #Devuelve un string con la informacion util del simulador
            return str(self.dia) + "," + str(self.hora) + "," + str(self.numpersonas) + "," +str(self.numedificios)  
        def actualizar_catalogo(self):#actualizamos los datos de las personas ordenadas por id
            nuevoCatalogo=[]
            for i in range(self.numedificios):
                for j in range (len(self.ciudad[i])):
                    nuevoCatalogo.append(self.ciudad[i][j]) #primero cogemos todas las personas sin ordenar
                    self.CatalogoPersonas = sorted(nuevoCatalogo, key=lambda x: x.idpersona, reverse=False)
            
        #Hace que el simulador avance una hora , con sus consecuencias en las personas
        def pasar_tiempo(self,mediaincubacion,desvincubacion,mediaduracion,desvduracion):
            if ((self.hora % 8)==0): #barajar a las personas cada 8 horas( hacen shuffle)
                self.moverpersonas2()
                print("owo")
            for j in range(len(self.ciudad)): #recorre los edificios 
                for i in range(len(self.ciudad[j])): #recorre las personas 
                    self.ciudad[j][i].infectar(0.4,self.ciudad,mediaincubacion,desvincubacion,self.dia) #pasa a infectar 
            if (self.hora<23):
                self.hora+=1
            else:#comienza un nuevo dia
                self.hora=0
                self.dia+=1
                for j in range(len(self.ciudad)):
                    for i in range(len(self.ciudad[j])):                     
                        self.ciudad[j][i].transicionEstados(self.dia, mediaincubacion,desvincubacion,mediaduracion,desvduracion)
                        
                for j in range(len(self.ciudad)):
                    for i in range(len(self.ciudad[j])-1,-1,-1):   
                        if (self.ciudad[j][i].estado=="muerto"):
                            
                            self.cementerio.append(self.ciudad[j].pop(i))
                            self.numpersonas-=1
                sorted(self.cementerio, key=lambda x: x.idpersona, reverse=False)
                            
        
        #Función que devuelve el total de personas sanas o que ya pasaron la enfermedad de la ciudad
        def personas_sanas(self):
            contador_sanos=0
            for j in range(len(self.ciudad)): #se recorre los edificios de la ciudad
                    for i in range(len(self.ciudad[j])): #se va recorriendo las personas de los edificios 
                        if self.ciudad[j][i].esta_sano():
                            contador_sanos+=1
            return contador_sanos
        
        def cogerpersona(self, idpersona):
            self.actualizar_catalogo()
            return self.CatalogoPersonas[idpersona]
        
        
        
        def moverpersonas(self):#Baraja a todas las personas de la ciudad indiscriminadamente
                                #Vamos a crear una nueva ciudad utilizando las personas actuales
            print(f"Total de vivos:{self.numpersonas}")
            personasColocadas=0
            edificiosSinVaciar=self.numedificios
            ciudadnueva=[] #la nueva ciudad
            for i in range(self.numedificios-1,-1,-1):
                if (len(self.ciudad[i])==0):
                    del self.ciudad[i]
                    edificiosSinVaciar-=1
            
            for i in range(self.numedificios):
                edificionuevo = [] #El edificio actual de la ciudad nueva.
                
                for j in range(self.capacidades[i]):
                    if personasColocadas==self.numpersonas:
                        break
                    
                    if edificiosSinVaciar>1:
                        edificioescogido=random.randint(0,edificiosSinVaciar)
                    else:
                        edificioescogido=0
                    edificionuevo.append(self.ciudad[edificioescogido].pop(0))
                    edificionuevo[j].lugar=i
                    personasColocadas+=1
                    if (personasColocadas==self.numpersonas):
                        break
                    if (len(self.ciudad[edificioescogido])==0):
                        del  self.ciudad[edificioescogido]
                        edificiosSinVaciar-=1
            
                ciudadnueva.append(edificionuevo)
            self.ciudad=ciudadnueva
            
        def moverpersonas2(self):
            print(f"Total de vivos:{self.numpersonas}")
            personasColocadas=0
            edificiosAntiguos=self.numedificios
            ciudadnueva=[] #la nueva ciudad
            opciones=[]
            for i in range(self.numedificios):
                edificionuevo=[]
                ciudadnueva.append(edificionuevo)
                opciones.append(i)
            
            personasColocadas=0
            edificiosLlenos=self.numedificios
            
            for i in range(self.numedificios):
                for j in range(len(self.ciudad[i])):
                    if (len(self.ciudad[i])==0):
                        break
                    edificioALlenar=random.choice(opciones)
                    ciudadnueva[edificioALlenar].append(self.ciudad[i].pop(0))
                    if (len(ciudadnueva[edificioALlenar])==self.capacidades[edificioALlenar]):
                        opciones.remove(edificioALlenar)
            self.ciudad=ciudadnueva
            
            
        #Función que contagia a la primera persona (la deja sintomática), se usa para crear pacientes 0
        def contagio_fijo(self):
            self.ciudad[0][0].estado=self.ciudad[0][0].estadosposibles[3]
            self.ciudad[0][0].cambioEstado=15
            
            
        def mostrarcementerio(self):#pues eso
            print(f"Total de muertos:{len(self.cementerio)}")
            for j in range(len(self.cementerio)):
                print(self.cementerio[j])

In [387]:
#Funcion que se encarga de generar la edad de manera aleatoria
def generador_edad():
        n = random.randint(0,110) #el rango de edad, sera desde los 0 años hasta los 110
        return n


In [388]:
#Funcion que se encarga de generar numeros aleatorios 
def generador_numeros(longitud):
    numero=0
    for i in range(0,longitud): 
        n = random.randint(0,9)
        numero=10*numero+n
    return numero


In [389]:
#Funcion que se encarga de crear una secuencia valida de DNI
#Para ello, se ha utilizado el algoritmo que se usa en la vida real -en cuanto a la relacion numero y letra- 
def creaDni():
    letras = "TRWAGMYFPDXBNJZSQVHLCKEO" #conjunto de letras del dni
    input=generador_numeros(8)
    #Se consigue que toda la secuencia de numeros sea 1 solo
    
    #Se calcula la letra que ha de ser adicionada
    valor =int(input / 23)
    #print(f"El valor en este caso es: {valor}")
    valor *= 23
    valor = input - valor;
    #Ahora, ha de pasarse el numero de la secuencia
    #a una lista donde cada elemento es un numero de la secuencia 
    DNI=str(input)+letras[valor]
    
    return DNI

In [390]:
#Esta función crea la ciudad en la versión 0 de la simulacion.
#La ciudad es una lista de edificios y cada edificio es una lista de personas de tamaño variable.
#Input: personas de la ciudad, media de personas por edificio.
#Output: la ciudad.
def crearciudadv0(numpersonas,personasedificio):
    edificios = [] #La ciudad.
    capacidadusada=0 #Numero de personas metidas en la ciudad.
    edificioactual=0
    while (capacidadusada < numpersonas):
        column = [] #El edificio.
        esteedificio=random.randint(1,personasedificio*3) 
        '''
        Maximo de personas en el edificio. 
        Si usamos desviación tipica de capacidad de edificio, iria aqui
        '''
        contadoredificio=0 #numero de personas colocadas en este edificio
        #print(f"edificio actual:{edificioactual}")
        
        #Bucle en el que se llena el edificio de personas.
        while (capacidadusada < numpersonas) and (contadoredificio < esteedificio):
            
            #print(f"contador del edificio:{contadoredificio}")
            #print(f"capacidad:{esteedificio}")
            
            #Insercion de persona.
            column.append(Persona(capacidadusada,generador_edad(),edificioactual, creaDni()))
            capacidadusada+=1
            contadoredificio+=1
            
            
        #Insercion de edificio en la ciudad
        edificios.append(column)
        edificioactual+=1
        
    return edificios
def printearciudad(ciudad):        
    for j in range(len(ciudad)):
        for i in range(len(ciudad[j])):
            print(ciudad[j][i])
        print()        
        
#Funcion que dada la ciudad, y un indice, permite imprimir el edificio que corresponda a ese indice
def printearEdificio(ciudad , i):
    
    for j in range(len(ciudad[i])):
        print(ciudad[i][j])
    

In [394]:
MiSimulador=Simulador(2000,35) #numPersonas, personasEdificio?? 
print(MiSimulador)
MiSimulador.contagio_fijo() #se crea un paciente que resulta ser el 0 
for i in range(1200):
    '''
    Datos inventados:
    MediaIncubación:7 dias
    DesvIncubación:3 dias
    MediaDuracion:14 dias
    DescDuracion:5 dias
    '''
    MiSimulador.pasar_tiempo(7,3,14,5)
    if MiSimulador.hora==0:
        print("--------")
        print(" ")
        print(" ")
        print(MiSimulador.dia)
        print(MiSimulador.capacidades)
        print(MiSimulador)
        printearciudad(MiSimulador.ciudad)
        print(MiSimulador.personas_sanas())
        print(" ")
        print(" ")
        print("--------")
        print(" ")
        print(" ")
        MiSimulador.mostrarcementerio()
        print(" ")
        print(" ")
        print("--------")

[ 59   4  64  26  21  51  62  77  22  99  85  89  41 102  55   3  97  52
  93  90  63  59 101   1  73   7  57 104   8  15 101  50  38  55   7  65
   4]
0,0,2000,37
Total de vivos:2000
owo
Total de vivos:2000
owo
Total de vivos:2000
owo
--------
 
 
1
[ 59   4  64  26  21  51  62  77  22  99  85  89  41 102  55   3  97  52
  93  90  63  59 101   1  73   7  57 104   8  15 101  50  38  55   7  65
   4]
1,0,2000,37
671,sano,12,20380146F,91
1906,sano,33,76747380Z,98
825,incubando,14,76021352G,87
1105,sano,19,74527350J,67
763,sano,13,6761527X,1
485,incubando,10,14876224P,25
1382,incubando,22,48872728M,42
993,sano,17,84223374L,49
1746,sano,30,78178734D,99
782,sano,13,44844684C,108
1769,sano,30,82667708G,89
1623,sano,27,34726443P,13
29,sano,0,54877418Q,94
160,sano,4,83883041V,5
1537,sano,26,44450581E,53
1002,sano,17,16508672P,89
1942,incubando,35,82318850X,94
1945,sano,35,66231618J,90
1311,sano,21,376676M,78
469,sano,9,76010027H,40
1977,sano,35,58255588S,8
49,sano,0,78101020N,60
141,sano,3,418

1251,sano,20,12836414E,91
940,sano,16,50647007H,85
1546,sano,26,55255600D,2
275,sano,6,72528656L,62
1317,sano,22,83815463J,7
1874,sano,33,22064665Y,25
378,sano,8,20623104Q,0
425,sano,9,64315311C,85
529,sano,10,70348041B,18
1687,sano,30,52787766Y,13
1820,sano,31,51343430R,66
1352,sano,22,68677028V,30
741,sano,13,12208780N,103
650,sano,11,30058545Y,82
1698,sano,30,45883376D,21
982,sano,17,31758605K,11
1898,sano,33,32708222Z,88
451,sano,9,35140806A,46
1907,sano,33,30634445D,36
190,sano,5,41153664V,92
960,sano,17,24810862A,15
1294,sano,21,68726571H,29
431,sano,9,48504475M,80
1600,sano,27,47212440H,29
1787,sano,31,41085448L,61
1615,sano,27,16304024Z,8
4,sano,0,38000783E,1
957,sano,17,82233515M,10
1219,sano,20,31634078Q,57
841,sano,14,28585235F,32
386,sano,9,68200271G,49
554,sano,10,66515831S,17
630,sano,11,53652614P,7
875,sano,16,88121847S,13

1063,sano,18,683671L,38
1161,incubando,19,40828413D,20
1677,sano,29,78336847C,90
631,sano,11,82882382L,74
1640,sano,27,61466152D,66
1190,sano,19,1586

456,sano,9,86415700P,43
630,sano,11,53652614P,7
378,sano,8,20623104Q,0
864,sano,16,33822305T,52
1488,sano,24,56177288H,73
1213,sano,20,26833818V,56
1556,sano,27,46482841W,38
1331,sano,22,53075352T,45
433,sano,9,31363417L,55
369,sano,8,26008038Y,53
1635,sano,27,15550077F,75
1603,sano,27,65552000E,27
1029,sano,18,52345283C,60
1944,sano,35,70606364K,71
1508,sano,26,52614828M,65
428,sano,9,38870564N,1
1339,sano,22,83186610A,7
833,sano,14,14400237Y,94
1191,sano,19,18871124S,88
263,sano,6,73863221D,34
1111,sano,19,78883735Z,44
717,sano,13,30854720J,17

1355,sano,22,8654664V,89
1631,sano,27,58725600E,62
681,sano,12,64811612G,25
1865,sano,32,61857331G,77

1958
 
 
--------
 
 
Total de muertos:0
 
 
--------
Total de vivos:2000
owo
Total de vivos:2000
owo
Total de vivos:2000
owo
--------
 
 
3
[ 59   4  64  26  21  51  62  77  22  99  85  89  41 102  55   3  97  52
  93  90  63  59 101   1  73   7  57 104   8  15 101  50  38  55   7  65
   4]
3,0,2000,37
1743,sano,30,84457151R,103
1419,incuban

1802,sano,31,46381630Z,98
105,sano,2,88575488G,52
1030,sano,18,86253578J,97
1939,sano,35,83313735F,63
814,sano,14,16311325R,44
1972,sano,35,78164374R,75
274,sano,6,64642473F,56
446,incubando,9,32246102X,39
497,incubando,10,87871735M,31
75,sano,2,56468811Q,46
1834,sano,32,36381214K,48
271,sano,6,8175614B,26
184,sano,5,6536602W,95
1406,sano,22,54888834R,25
985,sano,17,43181222X,48
1657,sano,28,43772063G,9
963,sano,17,21755270F,7
1073,sano,18,21303425C,52
227,incubando,6,53743753K,27
828,incubando,14,6140263E,34
1715,sano,30,33726634D,0
1749,sano,30,62860215L,33
17,sano,0,78254004T,28
1557,sano,27,10562608L,64
600,sano,11,21132662D,55
355,sano,7,32657504B,42
1564,sano,27,50217106X,15
431,sano,9,48504475M,80
27,sano,0,42724427H,105
799,sano,13,38621080D,6
1176,sano,19,31150346C,34
582,sano,11,78554626N,14
879,sano,16,17230051S,54
898,sano,16,82854155J,88
682,sano,12,55215811X,72
633,sano,11,51144800E,88
679,sano,12,37675034E,62
388,sano,9,10740841W,11
1120,incubando,19,32382538X,59
630,san

56,sano,0,57180058B,61
1458,sano,24,55800613Z,106
1922,sano,33,74420766B,36
919,incubando,16,28860544Y,23
1906,sano,33,76747380Z,98
28,sano,0,1672755B,19
1752,sano,30,72320767G,70
431,sano,9,48504475M,80
434,incubandoContagioso,9,50722714D,14
611,sano,11,80357723R,62
1601,sano,27,80722505A,89
1758,sano,30,67118565B,59
377,incubando,8,63843555V,95
941,incubando,16,6181860N,21
116,incubando,2,25212674M,62
1614,sano,27,36458215H,55
1893,sano,33,58788630D,33
1131,sano,19,71456707P,60
1505,sano,26,77881316G,37
277,sano,6,55580282T,62
54,sano,0,81135178X,4
773,sano,13,40527662Y,28
1552,sano,26,50675861F,21
1638,incubando,27,62536424E,102
270,sano,6,66041153B,28
962,sano,17,35713413W,104

1233,sano,20,24376614H,71
540,sano,10,38814778R,24
243,sano,6,12711634V,106
538,sano,10,47366765J,26
687,sano,12,10318134N,51
1680,sano,30,84600144A,81
600,sano,11,21132662D,55
923,sano,16,17060800K,41
1115,sano,19,16482484V,20
1015,incubando,18,42018477F,95
84,sano,2,1233433N,72
664,sano,12,51672173M,75
256

...

9,inmune,0,51838206R,26
163,inmune,4,67153561R,41
71,inmune,2,76482255X,49
269,inmune,6,38034187F,59
1435,inmune,24,82885681Y,62
1988,inmune,35,62683124M,78
868,inmune,16,37827674B,101
1004,inmune,17,4615405H,90
1426,inmune,24,71383160S,45
894,inmune,16,46084663T,13
553,inmune,10,78487601D,107
1770,inmune,30,77172307Q,28
29,inmune,0,54877418Q,94
937,inmune,16,30061418G,45
1413,inmune,22,30178570V,37
1560,inmune,27,57248717S,40
779,inmune,13,43653306L,82
1615,inmune,27,16304024Z,8
1818,inmune,31,52352612N,23
942,inmune,16,56587345P,34
1624,inmune,27,75063038P,14
185,inmune,5,45501505F,78
1337,inmune,22,36007281E,93
1012,inmune,18,40282086R,106
1379,inmune,22,26587040Y,26
479,inmune,9,1457104P,109
675,inmune,12,57122888L,55
617,inmune,11,20260076C,12
179,inmune,5,35188110L,45
839,inmune,14,61322742G,74
1533,inmune,26,23671642L,106
761,inmune,13,37026314V,103
1913,inmune,33,80058520M,97
962,inmune,17,35713413W,104
1049,inmune,18,85015318G,70
1751,inmune,30,41336374S,93
1386,inmune,22,6340

1790,inmune,31,3834484Q,62
274,inmune,6,64642473F,56
956,inmune,16,75233428Z,58
564,inmune,10,14432200E,42
258,inmune,6,7663010P,13
434,inmune,9,50722714D,14
934,inmune,16,88554716R,14
895,inmune,16,65466023L,77
1285,inmune,21,40603008G,9
155,inmune,4,27581858M,43
663,inmune,12,76504037B,69
1606,inmune,27,32601230H,105
1096,inmune,18,7434034C,2
1869,inmune,33,54542622P,25
1180,inmune,19,52335283W,106
544,inmune,10,65880608Y,54
1706,inmune,30,23255785W,14
1384,inmune,22,8006330F,11
277,inmune,6,55580282T,62
1987,inmune,35,38382488L,35
1798,inmune,31,46286588P,76
1579,inmune,27,8136666W,98

224,inmune,5,83275644G,51
661,inmune,12,35480216W,91
1888,inmune,33,48882632L,62
1148,inmune,19,2364578V,63
1487,inmune,24,72100575Z,87
1150,inmune,19,5360301J,63
869,inmune,16,37058255B,62
631,inmune,11,82882382L,74
20,inmune,0,58684062E,4
1449,inmune,24,45417455E,46
767,inmune,13,76378585R,14
1610,inmune,27,72575146A,103
1867,inmune,32,88267070Q,106
476,inmune,9,67438836F,101
137,inmune,3,8101512S,5

owo
Total de vivos:1639
owo
--------
 
 
49
[ 59   4  64  26  21  51  62  77  22  99  85  89  41 102  55   3  97  52
  93  90  63  59 101   1  73   7  57 104   8  15 101  50  38  55   7  65
   4]
49,0,1639,37
969,inmune,17,2411488F,68
536,inmune,10,47852311Y,86
795,inmune,13,84256263H,78
195,inmune,5,22721207J,36
179,inmune,5,35188110L,45
1246,inmune,20,22116444N,4
199,inmune,5,62855426Z,93
1585,inmune,27,44522513X,4
300,inmune,7,2015725M,61
1133,inmune,19,8511176A,22
1307,inmune,21,24233427Y,72
1682,inmune,30,14382588K,28
1625,inmune,27,58711413A,28
1478,inmune,24,72372711Z,25
1287,inmune,21,4161670G,50
511,inmune,10,78515426G,2
860,inmune,16,18185373D,23
920,inmune,16,1480118E,32
1156,inmune,19,72883536D,68
1711,inmune,30,65411872X,53
1751,inmune,30,41336374S,93
978,inmune,17,76000151D,50
881,inmune,16,12246056M,104
1370,inmune,22,56703321H,36
1332,inmune,22,3042813M,41
609,inmune,11,31560465A,16
403,inmune,9,7737151C,12
320,inmune,7,38710801F,87
867,inmune,16,61657201C,2
1764,inmune

945,muerto,16,47172320X,72
1792,muerto,31,21126358F,47
1036,muerto,18,17482267J,70
662,muerto,12,74862857L,81
1259,muerto,21,88765358D,67
1247,muerto,20,53050216A,23
949,muerto,16,3216121P,102
1509,muerto,26,7270455V,39
482,muerto,9,46405712S,88
201,muerto,5,53775480P,92
1388,muerto,22,75656183Y,65
877,muerto,16,84118448L,20
999,muerto,17,30867475A,34
810,muerto,14,67606832B,88
118,muerto,2,16486136N,21
695,muerto,12,37267446V,87
1638,muerto,27,62536424E,102
745,muerto,13,57078206A,96
1972,muerto,35,78164374R,75
1853,muerto,32,86568664E,108
1090,muerto,18,63620145Y,35
851,muerto,14,48534621K,85
1736,muerto,30,6661758S,44
1541,muerto,26,56775616R,98
910,muerto,16,44733215D,92
1779,muerto,30,61672237Z,101
1939,muerto,35,83313735F,63
571,muerto,11,68324665Z,94
1189,muerto,19,88163675Y,86
1622,muerto,27,11777215L,1
384,muerto,8,2658822E,77
374,muerto,8,21758663L,77
1269,muerto,21,63540376R,90
495,muerto,10,62355275K,75
912,muerto,16,45584463G,87
1390,muerto,22,65485086S,85
95,muerto,2,2873

1128,inmune,19,13823512Y,12
575,inmune,11,56286063A,16
1738,inmune,30,36144617W,61
1520,inmune,26,10422627Q,62
594,inmune,11,50063233F,8
1108,inmune,19,50865088J,65
1481,inmune,24,35164661F,59
766,inmune,13,78821006Y,19
1062,inmune,18,26251428X,3
263,inmune,6,73863221D,34
1824,inmune,31,24850610F,27
600,inmune,11,21132662D,55

1182,inmune,19,25324727W,4
239,inmune,6,18203887P,14
1893,inmune,33,58788630D,33
797,inmune,13,71711635G,21
197,inmune,5,88665120M,19
1749,inmune,30,62860215L,33
73,inmune,2,12584547M,6
46,inmune,0,64474720Q,45
683,inmune,12,77277538E,29
1068,inmune,18,82837545D,61
541,inmune,10,60724377F,16
1012,inmune,18,40282086R,106
704,inmune,13,40110407V,18
1369,inmune,22,75308277K,33
1757,inmune,30,61183031H,6
1499,inmune,26,70020634D,82
834,inmune,14,6130824J,93
62,inmune,1,38107524C,47
1746,inmune,30,78178734D,99
808,inmune,14,13384321T,38
1700,inmune,30,76067251H,32
1152,inmune,19,83822143T,34
552,inmune,10,27161554A,82
193,inmune,5,47426565J,109
100,inmune,2,17213846W,

In [323]:
printearEdificio(MiSimulador.ciudad,0)

761,asintomatico,0,44712543Z,41
651,asintomatico,0,38836458S,43
193,asintomatico,0,32314256S,28
550,sintomatico,0,47667568E,16
432,sintomatico,0,24304447W,92
870,asintomatico,0,74604356S,21
844,asintomatico,0,86485332L,73
175,sintomatico,0,50636048F,56
311,asintomatico,0,56742864R,57
873,incubandoContagioso,0,24084815C,84
362,asintomatico,0,36742254F,17
848,asintomatico,0,68453240L,83
30,sintomatico,0,48033012L,16
784,asintomatico,0,27222802W,78
36,incubandoContagioso,0,63356311M,33
359,asintomatico,0,68077808Q,82
660,incubandoContagioso,0,22183767Z,29
674,asintomatico,0,70441503R,71
33,asintomatico,0,45528108E,17
437,sintomatico,0,85412085K,109
888,sintomatico,0,26556562A,3
874,sintomatico,0,68826840F,34
153,asintomatico,0,48745675G,40
428,sintomatico,0,41404572H,31
391,sintomatico,0,50321368J,41
546,asintomatico,0,68112611C,100
361,asintomatico,0,1570532T,11
472,asintomatico,0,87283146D,21
510,sintomatico,0,46143821W,55
727,sintomatico,0,10618081Q,49
708,asintomatico,0,27580680T,93
4

In [340]:
printearciudad(MiSimulador.ciudad)

247,asintomatico,0,88204178Y,105
88,sintomatico,0,36071843T,24
197,asintomatico,0,50623185R,52
343,asintomatico,0,52872752F,86
16,asintomatico,0,33580276T,35
126,asintomatico,0,32625042W,41
200,asintomatico,0,62782272T,103
399,sintomatico,0,83080173X,109
358,sintomatico,0,30012326V,53
158,asintomatico,0,13267662C,94
273,sintomatico,0,62450417N,100
395,sintomatico,0,56464786Q,9
246,sintomatico,0,56227015L,25
153,sintomatico,0,34418838M,19
387,asintomatico,0,27476376R,90
298,asintomatico,0,27441310X,105
203,asintomatico,0,16332760T,106
56,asintomatico,0,22141215N,90
46,sintomatico,0,31205305D,49
351,sintomatico,0,61140157Q,104
177,asintomatico,0,83852621A,100
346,sintomatico,0,54051003Z,76
208,asintomatico,0,34354048Y,54
147,asintomatico,0,54713145D,87
15,asintomatico,0,70522455Q,33
125,asintomatico,0,6042748G,38
251,asintomatico,0,58672511V,78
81,asintomatico,0,16611415X,18
391,sintomatico,0,64685307S,30
155,asintomatico,0,40654814Z,75
314,sintomatico,0,47437185F,101
110,asintomatico,0,

In [149]:
MiSimulador.actualizar_catalogo()
for i in range(len(MiSimulador.CatalogoPersonas)):
                print(MiSimulador.CatalogoPersonas[i])
            

IndexError: list index out of range

In [218]:
personitaputa=MiSimulador.cogerpersona(0)
print(personitaputa.contadorInfecciones)

123


In [167]:
MiSimulador=Simulador(100,10) 
print(MiSimulador.capacidades)
print(len(MiSimulador.ciudad))

[23  1 29  7 22 16  2]
0,sano,0,87204200E,7
1,sano,0,81351675P,65
2,sano,0,7541462S,84
3,sano,0,2734837E,49
4,sano,0,67118311X,62
5,sano,0,63653820D,41
6,sano,0,56840305Z,104
7,sano,0,71438685H,88
8,sano,0,37235031D,2
9,sano,0,24211256F,28
10,sano,0,76816004Y,83
11,sano,0,25172814G,49
12,sano,0,75582883F,33
13,sano,0,65767330A,93
14,sano,0,22875655Q,88
15,sano,0,11256111A,85
16,sano,0,87500174D,37
17,sano,0,23444251Y,63
18,sano,0,50162442V,58
19,sano,0,52664477C,99
20,sano,0,10176827V,22
21,sano,0,57341107Z,72
22,sano,0,32625576F,67
23,sano,1,76036012J,70
24,sano,2,26170230W,84
25,sano,2,30847235A,11
26,sano,2,78242507A,47
27,sano,2,12318218Q,13
28,sano,2,3440412A,53
29,sano,2,32441324P,31
30,sano,2,26126301A,94
31,sano,2,28610526K,51
32,sano,2,27767452N,105
33,sano,2,50417272F,78
34,sano,2,2504088D,32
35,sano,2,57177760J,40
36,sano,2,31066118H,88
37,sano,2,76265586R,79
38,sano,2,68173116N,58
39,sano,2,35280872E,52
40,sano,2,34084064L,36
41,sano,2,6287818D,23
42,sano,2,11550463R,81
43,